# Практика. 9.5

Обучите модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты.

КРИТЕРИИ ОЦЕНИВАНИЯ:

*	Верно выделены три столбца-признака для обучения, выбранные `RFE`.
*	Верно выделены три столбца-признака для обучения, выбранные `SelectKBest`.
*	Обучена регрессия на первых трёх столбцах, оценено качество модели на тесте.
*	Обучена регрессия на вторых трёх столбцах, оценено качество модели на тесте.
*	Произведено сравнение выбранных метрик в форме комментария. Дан ответ на вопрос «Какой метод отбора признаков показал наилучший результат на тестовой выборке?» (в текстовой ячейке).

In [30]:
import pandas as pd

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error


In [31]:
data = pd.read_excel('data/data_ford_price.xlsx') 
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


In [32]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

### RFE

In [33]:
model = LinearRegression()
selector_rfe = RFE(model, n_features_to_select=3, step=1)
selector_rfe = selector_rfe.fit(X_train, y_train)
 
selector_rfe.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [34]:
X_train_rfe = X_train[selector_rfe.get_feature_names_out()]
X_test_rfe = X_test[selector_rfe.get_feature_names_out()]

model = model.fit(X_train_rfe, y_train)
y_rfe_pred = model.predict(X_test_rfe)
print(f'MAE RFE: {mean_absolute_error(y_test, y_rfe_pred):.2f}')

MAE RFE: 5206.68


### SelectKBest

In [35]:
selector_kbest = SelectKBest(f_regression, k=3)
selector_kbest.fit(X_train, y_train)
 
selector_kbest.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

In [36]:
X_train_kbest = X_train[selector_kbest.get_feature_names_out()]
X_test_kbest = X_test[selector_kbest.get_feature_names_out()]

model = model.fit(X_train_kbest, y_train)
y_kbest_pred = model.predict(X_test_kbest)
print(f'MAE SelectKBest: {mean_absolute_error(y_test, y_kbest_pred):.2f}')

MAE SelectKBest: 4800.92


### Вывод

На нашей тестовой выборке лучший результат показала методика отбора признаков на основе фильтров – `SelectKBest`.

***MAE*** (меньше – лучше): **4800.92** у `SelectKBest` против **5206.68** у `RFE`